Learning how binding works in langchain
youtube: https://www.youtube.com/watch?v=obg7vxXrSFc
medium notes: https://medium.com/@anubhavm48/practical-introduction-to-llm-tools-in-lang-chain-bb6b8b42d8b6

Tool creation in Langchain can be done in 2 ways, 
1 using decorator i,e @tool 
2. using StrucutredTool


In [1]:
from langchain_core.tools import StructuredTool

Creating 2 functions for tool usage

In [2]:
def add(a:int, b:int) -> int:
    return a+b

adder = StructuredTool.from_function(func=add)

ValueError: Function must have a docstring if description not provided.

Error generated because we need to provide fucntion docstring, based on the docstring LLM/LRM knows what that fucntion is doing

In [3]:
def add(a:int, b:int) -> int:
    """ Adds a and b """
    return a + b



def mul(a:int, b:int) -> int:
    """multiplies a and b """
    return a * b

adder = StructuredTool.from_function(func=add)
multiplier = StructuredTool.from_function(func=mul)

In [4]:
# creating tool list
tool_list = [adder, multiplier]

In [5]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
import os

In [6]:
llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b", api_key=os.getenv("GROQ_API_KEY"))

In [7]:
# now bind available tools to the llm
llm_with_tools = llm.bind_tools(tools=tool_list)

In [8]:
llm_with_tools.invoke("calculate addition of 5 and 10")

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, so I\'m trying to figure out how to add 5 and 10 using the tools provided. I see that there are two functions available: add and mul. Since I need to add numbers, the add function makes sense here. \n\nFirst, I\'ll need to call the add function with the arguments a=5 and b=10. I should make sure that the JSON structure is correct, including the function name and the arguments. Let me double-check the syntax to ensure there are no mistakes.\n\nI remember that the function call should be wrapped in <tool_call> tags and the arguments should be a JSON object. So, putting it all together, the final tool call should look like this:\n\n<tool_call>{"name":"add","arguments":{"a":5,"b":10}}</tool_call>\n\nI think that should do it. Hopefully, this will correctly add the two numbers and give me the result I need.\n', 'tool_calls': [{'id': '83xh7j27d', 'function': {'arguments': '{"a":5,"b":10}', 'name': 'add'}, 'type': 'function'

Now we will pass query to llm which has multiple steps to perform

In [15]:
query = "Multiply 2 with 5 and then, add 15 to it"

In [16]:
from langchain_core.messages import HumanMessage

In [34]:
# Initiating the messages list with the user query as the first element
messages = []
messages = [HumanMessage(query)]

In [35]:
ai_msg = llm_with_tools.invoke(messages) # NLP process via LLM
messages.append(ai_msg) # keeps track of all the messages like intital query and NLP processed msg

In [36]:
messages

[HumanMessage(content='Multiply 2 with 5 and then, add 15 to it', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Alright, so the user has given me a problem to solve: "Multiply 2 with 5 and then, add 15 to it." I need to figure out how to approach this using the tools provided. Let me break it down step by step.\n\nFirst, I need to understand the operations involved. The problem mentions two mathematical operations: multiplication and addition. The tools available are "add" and "mul," which correspond to addition and multiplication functions. That\'s perfect because the problem requires both operations.\n\nThe first part of the problem is multiplying 2 by 5. I remember that the multiplication function is called "mul" and it takes two parameters, a and b. So, I should call the "mul" function with a=2 and b=5. That should give me the product of 2 and 5.\n\nNext, after obtaining the result from the multiplication, I need to add

In [37]:
ai_msg.tool_calls

[{'name': 'mul',
  'args': {'a': 2, 'b': 5},
  'id': 'epzhqxtdh',
  'type': 'tool_call'}]

In [38]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": adder, "mul": multiplier}[tool_call["name"].lower()]
    selected_tool
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

print(messages)

[HumanMessage(content='Multiply 2 with 5 and then, add 15 to it', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'reasoning_content': 'Alright, so the user has given me a problem to solve: "Multiply 2 with 5 and then, add 15 to it." I need to figure out how to approach this using the tools provided. Let me break it down step by step.\n\nFirst, I need to understand the operations involved. The problem mentions two mathematical operations: multiplication and addition. The tools available are "add" and "mul," which correspond to addition and multiplication functions. That\'s perfect because the problem requires both operations.\n\nThe first part of the problem is multiplying 2 by 5. I remember that the multiplication function is called "mul" and it takes two parameters, a and b. So, I should call the "mul" function with a=2 and b=5. That should give me the product of 2 and 5.\n\nNext, after obtaining the result from the multiplication, I need to add 

In [39]:
final_response = llm_with_tools.invoke(messages)

In [40]:
final_response

AIMessage(content='The result of multiplying 2 by 5 is 10. Adding 15 to this result gives us 25. \n\n**Answer:** 25', additional_kwargs={'reasoning_content': "Alright, let's tackle this problem step by step. The user has asked me to multiply 2 by 5 and then add 15 to the result. \n\nFirst, I need to understand what's being asked. Multiplication and addition are basic arithmetic operations, so it should be straightforward. I should make sure I perform the operations in the correct order as specified.\n\nStarting with the multiplication part: 2 multiplied by 5. I know that 2 times 5 equals 10. So, that's the first step done.\n\nNext, I need to add 15 to the result of the multiplication. The multiplication gave me 10, so now I add 15 to that. Adding 15 to 10 gives me 25.\n\nI should double-check my calculations to ensure there are no mistakes. Multiplying 2 by 5 is indeed 10, and adding 15 to 10 is 25. Everything checks out.\n\nIt's important to present the answer clearly, so the user can

In [41]:
final_response.content

'The result of multiplying 2 by 5 is 10. Adding 15 to this result gives us 25. \n\n**Answer:** 25'